In [13]:
import pandas as pd
import numpy as np
import os
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql import types as types

In [23]:
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format


config = configparser.ConfigParser()
config.read('dl.cfg')

os.environ['AWS_ACCESS_KEY_ID']=config['AWS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY']=config['AWS']['AWS_SECRET_ACCESS_KEY']

In [14]:
from pyspark.sql.types import StructType as R,TimestampType as Ts ,StructField as Fld,FloatType as Flt ,DoubleType as Dbl, StringType as Str, IntegerType as Int, DateType as Date

In [27]:
spark = SparkSession \
        .builder \
        .appName("Data Wrangling with Spark SQL") \
       	.config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()

In [28]:
INPUT_BUCKET='s3a://udacity-dend/'
LOG_FILE_PATH='log-data/*/*/*.json'
input_log_data = os.path.join(INPUT_BUCKET, LOG_FILE_PATH)
### Create Schema
logSchema = R([
    Fld("artist",Str()),
    Fld("auth",Str()),
    Fld("first_name",Str()),
    Fld("gender",Str()),
    Fld("item_in_session",Int()),
    Fld("last_name",Str()),
    Fld("length",Flt()),
    Fld("level",Str()),
    Fld("location",Str()),
    Fld("method",Str()),
    Fld("page",Str()),
    Fld("registration",Flt()),
    Fld("session_id",Int()),
    Fld("song",Str()),
    Fld("status",Int()),
    Fld("ts",Ts()),
    Fld("user_agent",Str()),
    Fld("user_id",Int())
    ])

In [29]:
log_data = spark.read.json(input_log_data, schema=logSchema)

Py4JJavaError: An error occurred while calling o231.json.
: com.amazonaws.AmazonClientException: Unable to load AWS credentials from any provider in the chain
	at com.amazonaws.auth.AWSCredentialsProviderChain.getCredentials(AWSCredentialsProviderChain.java:117)
	at com.amazonaws.services.s3.AmazonS3Client.invoke(AmazonS3Client.java:3521)
	at com.amazonaws.services.s3.AmazonS3Client.headBucket(AmazonS3Client.java:1031)
	at com.amazonaws.services.s3.AmazonS3Client.doesBucketExist(AmazonS3Client.java:994)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.initialize(S3AFileSystem.java:297)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:2669)
	at org.apache.hadoop.fs.FileSystem.access$200(FileSystem.java:94)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:2703)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:2685)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:373)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:295)
	at org